In [1]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


## General Settings

In [2]:
#@markdown Name/Path of the initial model.
MODEL_NAME = "CompVis/stable-diffusion-v1-4" #@param {type:"string"}

#@markdown Path for images of the concept for training.
INSTANCE_DIR = "/content/data/sks" #@param {type:"string"}
!mkdir -p $INSTANCE_DIR

#@markdown A general name for class like dog for dog images.
CLASS_NAME = "man" #@param {type:"string"}
CLASS_DIR = f"/content/data/{CLASS_NAME}"

#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = True #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown Enter the directory name to save model at.

OUTPUT_DIR = "stable_diffusion_weights/sks" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

#@markdown sks is a rare identifier, feel free to replace it.

Mounted at /content/drive
[*] Weights will be saved at /content/drive/MyDrive/stable_diffusion_weights/sks


https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [3]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 31.9 MB/s 
     |████████████████████████████████| 18.7 MB 144 kB/s 
     |████████████████████████████████| 143 kB 34.1 MB/s 
     |████████████████████████████████| 5.3 MB 45.1 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 62.5 MB 1.2 MB/s 
     |████████████████████████████████| 5.3 MB 63.8 MB/s 
     |████████████████████████████████| 7.6 MB 65.0 MB/s 
     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 2.3 MB 57.7 MB/s 
     |████████████████████████████████| 270 kB 73.8 MB/s 
     |████████████████████████████████| 112 kB 76.2 MB/s 
     |████████████████████████████████| 212 kB 64.6 MB/s 
     |██████

In [4]:
# #@title Login to HuggingFace 🤗

# #@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
# from huggingface_hub import notebook_login
# !git config --global credential.helper store
# notebook_login()

### Install xformers from precompiled wheel.

In [5]:
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

# If precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

     |████████████████████████████████| 102.9 MB 95 kB/s 


## Inference

In [6]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

model_path = OUTPUT_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
g_cuda = None

In [7]:
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 50021 #@param {type:"number"}
g_cuda.manual_seed(seed)

In [8]:
# #@title Run for generating images.

# prompt = "masterwork portrait of sks man, painting by van gogh" #@param {type:"string"}
# negative_prompt = "" #@param {type:"string"}
# num_samples = 8 #@param {type:"number"}
# guidance_scale = 7.5 #@param {type:"number"}
# num_inference_steps = 50 #@param {type:"number"}
# height = 512 #@param {type:"number"}
# width = 512 #@param {type:"number"}

# with autocast("cuda"), torch.inference_mode():
#     images = pipe(
#         prompt,
#         height=height,
#         width=width,
#         negative_prompt=negative_prompt,
#         num_images_per_prompt=num_samples,
#         num_inference_steps=num_inference_steps,
#         guidance_scale=guidance_scale,
#         generator=g_cuda
#     ).images

# for img in images:
#     display(img)

# Test section

## define temp functions

In [9]:
from datetime import datetime

def get_string_timestamp():
  return datetime.now().strftime("%m%d%Y_%H_%M_%S")

In [10]:
from tqdm.auto import tqdm

In [11]:
#get images for all global variables and a single prompt
def get_images_for_prompt(curr_prompt, height, width, negative_prompt, num_samples, num_inference_steps, guidance_scale, g_cuda, display_images_in_notebook):
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          curr_prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_images_per_prompt=num_samples,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images

  cpt = 0
  for img in tqdm(images):
    cpt = cpt + 1
    if display_images_in_notebook:
      display(img)
    img.save(output_pictures_folder+get_string_timestamp()+'_'+str(cpt)+'_image.png')

In [12]:
output_pictures_folder = '/content/drive/MyDrive/AI/DREAMBOOTH/output/'

## archive images


In [4]:
output_pictures_folder_bash = '/content/drive/MyDrive/AI/DREAMBOOTH'
!mkdir $output_pictures_folder_bash/$(date +%Y%m%d_%H%M%S) && mv $output_pictures_folder_bash/output/*.png $output_pictures_folder_bash/$(date +%Y%m%d_%H%M%S)

## small run section

In [31]:
#define a prompt list
master_prompt_list = [
    # 'award winning photorealistic portrait of sks man, painting by van gogh and Jonathan Yeo, trending on Deviantart',
    # 'award winning photorealistic masterwork portrait of sks man, painting by van gogh and Jonathan Yeo, trending on Deviantart',
    # 'sks man portrait, painting by van gogh and Jonathan Yeo, trending on Deviantart, award winning photorealistic masterwork portrait',
    # 'sks man portrait, painting by van gogh and Jonathan Yeo, trending on Deviantart, award winning photorealistic portrait',
    # 'sks man portrait, painting by van gogh and Jonathan Yeo, trending on Deviantart, award winning masterwork portrait',
    # 'sks man portrait, painting by van gogh and Jonathan Yeo, trending on Deviantart, photorealistic masterwork portrait',
    # 'sks man sks man award winning photorealistic masterwork portrait of sks man, painting by van gogh and Jonathan Yeo, trending on Deviantart',
    # 'sks portrait, Postimpressionism masterwork painting of sks man, trending on Deviantart'
    # 'award winning masterwork portrait of sks man, painting by van gogh and Jonathan Yeo, trending on Deviantart',
    # 'masterwork portrait of sks man, painting by van gogh and Jonathan Yeo, trending on Artstation',
    # 'masterwork portrait of sks man, painting by van gogh, trending on Artstation',
    # 'masterwork portrait of sks man, painting in the style of van gogh',
    # 'masterwork portrait of sks man by van gogh',
    # 'sks man portrait in the Dutch Post-Impressionist style, sks man portrait oil on canvas by van gogh'

    
    # "sks man portrait, highly detailed portrait of sks man, unreal engine, fantasy art by greg rutkowski, loish, rhads, ferdinand knab, makoto shinkai and lois van baarle, ilya kuvshinov, rossdraws, tom bagshaw, global illumination, radiant light, detailed and intricate environment",
    # "sks man portrait, highly detailed portrait of sks man, unreal engine, fantasy art by van gogh, loish, rhads, ferdinand knab, makoto shinkai and lois van baarle, ilya kuvshinov, rossdraws, tom bagshaw, global illumination, radiant light, detailed and intricate environment",
    # "sks man portrait, highly detailed portrait of sks man, unreal engine, fantasy art by Jonathan Yeo, loish, rhads, ferdinand knab, makoto shinkai and lois van baarle, ilya kuvshinov, rossdraws, tom bagshaw, global illumination, radiant light, detailed and intricate environment",
    
    # "sks man portrait, high quality high detail painting by rembrandt, portrait of a modern sks man, hd, photorealistic lighting",
    # "sks man portrait, high quality high detail painting by van gogh, portrait of a modern sks man, hd, photorealistic lighting",
    # "sks man portrait, high quality high detail painting by greg rutkowski, portrait of a modern sks man, hd, photorealistic lighting",
    # "sks man portrait, high quality high detail painting by Jonathan Yeo, portrait of a modern sks man, hd, photorealistic lighting",

    # "sks man portrait, high quality high detail sks man portrait, portrait of a modern sks man, hd, photorealistic lighting, oil on canvas masterwork portrait painting",

    # "sks man portrait in byzantine mosaic",
    # "sks man portrait in Opus tessellatum mosaic", 
    # "sks man portrait in roman mosaic", 
    # "sks man portrait in intricate mosaic", 
    # "sks man portrait in gilded mosaic", 

    # "sks man portrait in the style of a roman mosaic", 
    # "sks man roman mosaic", 
    # "sks man, roman mosaic", 

    # "sks man mosaic portrait of emperor sks man by klimt and gaudí from roman villa pompeii high detail 4k picture museum of mosaics athens intricate tesselation sks man, mosaic beard, mosaic", 

    # "Opus tessellatum of sks man, 4k picture of mosaic sks man portrait",


    # "award winning picture portrait of sks man face by Martin Schoeller and Steve McCurry, with giant fire explosion in the background",
    # "award winning picture portrait of sks man face by Martin Schoeller, with giant fire explosion in the background",
    # "award winning picture portrait of sks man face by Steve McCurry, with giant fire explosion in the background",

    # "award winning picture portrait of sks man face with giant fire explosion in the background",
    # "award winning picture portrait of sks man face giant fire background",
    # "award winning picture portrait of sks man face giant explosion background",

    # "award winning selfie of sks man in the middle of a forest fire",
    # "selfie sks man in active forest fire",
    # "selfie sks man in raging forest fire",
    
    # "selfie sks man in the worst forest fire",
    # "award winning selfie of sks man forest fire in the background",

    # "HD selfie sks man white male in front of nuclear missile in a secret iranian facility with Radiation warning symbol sign",

  # "HD selfie sks man inside of United States Capitol during attack",
  # "HD selfie sks man inside of United States Capitol during riots",
  # "HD selfie sks man participating in riots inside the United States Capitol",

  # "HD selfie sks man participating in riots inside the USA Capitol",

  # "sks man white male wearing a red baseball cap",
  # "portrait sks man white male wearing a red baseball cap",
  # "picture sks man white male wearing a red baseball cap",
  # "picture portrait sks man white male wearing a red baseball cap",


# "HD selfie sks man red baseball cap",

  # "HD selfie sks man red baseball cap inside of building engulfed in massive flames smoke column",

  # "flames engulfing usa capitol",

  "HD selfie sks man wearing apollo spacesuit",
  "HD selfie sks man wearing Extravehicular Mobility Unit",
  "HD selfie sks man wearing E.M.U",
  "HD selfie sks man wearing NASA EMU",
  "HD selfie sks man wearing NASA EMU performing EVA on the surface of mars",
  "HD selfie sks man wearing NASA spacesuit performing EVA on the surface of mars",
  "HD selfie sks man wearing NASA EMU spacesuit performing EVA on the surface of mars",

  # "government building burning",
  # "building in flames",
]

In [32]:
# craft age related prompts

# master_prompt_list = []
# for age in range(15, 101, 5):
#   # master_prompt_list.append("HD selfie "+str(age)+" year old sks man")
#   master_prompt_list.append(f"HD selfie {age} year old sks man")
# # master_prompt_list

In [33]:
#pass settings as global variables
negative_prompt = "blurry" #@param {type:"string"}
num_samples = 8 #@param {type:"number"}
guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

In [ ]:
for curr_prompt in tqdm(master_prompt_list):
  for gens in tqdm(range(20)):

    display_images_in_notebook = False

    get_images_for_prompt(curr_prompt, height, width, negative_prompt, num_samples, num_inference_steps, guidance_scale, g_cuda, display_images_in_notebook)